In [0]:
# Databricks notebook source
# DBTITLE 1,Importação de Bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, year, month, quarter, to_date

# DBTITLE 1,Criação da SparkSession
spark = SparkSession.builder.appName("CamadaSilver").getOrCreate()

# DBTITLE 1,Carregar Dados da Camada Bronze
inadimplencia_bronze_df = spark.read.option("delimiter", ";").csv('dbfs:/mnt/Aneel/Bronze/inadimplencia.csv', header=True, inferSchema=True)
desemprego_bronze_df = spark.read.option("delimiter", ";").csv('dbfs:/mnt/Aneel/Bronze/taxa_desemprego.csv', header=True, inferSchema=True)
pib_bronze_df = spark.read.option("delimiter", ";").csv('dbfs:/mnt/Aneel/Bronze/pib_bcdata.sgs.1207.csv', header=True, inferSchema=True)
inflacao_bronze_df = spark.read.option("delimiter", ";").csv('dbfs:/mnt/Aneel/Bronze/inflacao_ipca.csv', header=True, inferSchema=True)
dominio_indicadores_bronze_df = spark.read.option("delimiter", ",").csv('dbfs:/mnt/Aneel/Bronze/dominio-indicadores.csv', header=True, inferSchema=True)

# DBTITLE 1,Verificar Dados Carregados na Camada Bronze
print("Dados da Camada Bronze")
print("======================")
print("Tabela de Inadimplência")
display(inadimplencia_bronze_df.limit(5))
print("Tabela de Desemprego")
display(desemprego_bronze_df.limit(5))
print("Tabela de PIB")
display(pib_bronze_df.limit(5))
print("Tabela de Inflação")
display(inflacao_bronze_df.limit(5))

# DBTITLE 1,Renomear e Limpar Colunas de Dados Econômicos
# Ajustar as tabelas de dados econômicos para garantir que todas possuam a coluna 'AnoIndice'
desemprego_bronze_df = desemprego_bronze_df.withColumnRenamed('Ano', 'AnoIndice').withColumnRenamed('Media', 'TaxaDesemprego')
pib_bronze_df = pib_bronze_df.withColumnRenamed('Data', 'AnoIndice').withColumnRenamed('valor', 'PIB')
inflacao_bronze_df = inflacao_bronze_df.withColumnRenamed('Ano', 'AnoIndice').withColumnRenamed('Media', 'Inflacao')

# Ajustar o formato da coluna AnoIndice na tabela de PIB
pib_bronze_df = pib_bronze_df.withColumn('AnoIndice', year(to_date(col('AnoIndice'), 'dd/MM/yyyy')))

# Limpar os dados para remover possíveis caracteres que possam impedir a conversão para float
desemprego_bronze_df = desemprego_bronze_df.withColumn("TaxaDesemprego", regexp_replace(col("TaxaDesemprego"), ",", "."))
pib_bronze_df = pib_bronze_df.withColumn("PIB", regexp_replace(col("PIB"), ",", "."))
inflacao_bronze_df = inflacao_bronze_df.withColumn("Inflacao", regexp_replace(col("Inflacao"), ",", "."))

# Converter colunas de string para float
desemprego_bronze_df = desemprego_bronze_df.withColumn("TaxaDesemprego", col("TaxaDesemprego").cast("float"))
pib_bronze_df = pib_bronze_df.withColumn("PIB", col("PIB").cast("float"))
inflacao_bronze_df = inflacao_bronze_df.withColumn("Inflacao", col("Inflacao").cast("float"))

# Ajuste dos Dados de Inadimplência
inadimplencia_bronze_df = inadimplencia_bronze_df.withColumn("VlrIndiceEnviado", regexp_replace(col("VlrIndiceEnviado"), ",", "."))

# Converter VlrIndiceEnviado para float
inadimplencia_bronze_df = inadimplencia_bronze_df.withColumn("VlrIndiceEnviado", col("VlrIndiceEnviado").cast("float"))

# Converter DatGeracaoConjuntoDados para data
inadimplencia_bronze_df = inadimplencia_bronze_df.withColumn("DatGeracaoConjuntoDados", to_date(col("DatGeracaoConjuntoDados"), 'dd-MM-yyyy'))

# Verificar os dados ajustados de inadimplência
print("Dados ajustados de inadimplência - Converter DatGeracaoConjuntoDados para data")
display(inadimplencia_bronze_df.limit(5))

# DBTITLE 1,Selecionar e Transformar Dados de Inadimplência para a Camada Silver
inadimplencia_silver_df = inadimplencia_bronze_df.select(
    'DatGeracaoConjuntoDados', 'SigAgente', 'NumCNPJ', 'SigIndicador', 'AnoIndice', 'NumPeriodoIndice', 'VlrIndiceEnviado'
)

# Verificar os dados ajustados de inadimplência na camada Silver
display(inadimplencia_silver_df.limit(5))

# DBTITLE 1,Realizar a Junção das Tabelas Econômicas
economicos_silver_df = (desemprego_bronze_df
    .join(pib_bronze_df, 'AnoIndice')
    .join(inflacao_bronze_df, 'AnoIndice')
    .select('AnoIndice', 'TaxaDesemprego', 'Inflacao', 'PIB'))

# Verificar os dados da junção
print("Dados Economicos - Após Junção das Tabelas")
display(economicos_silver_df.limit(5))

# DBTITLE 1,Selecionar e Transformar Dados de Domínio de Indicadores
dominio_indicadores_silver_df = dominio_indicadores_bronze_df.select(
    'SigIndicador', 'DscIndicador'
)
# Renomear colunas
dominio_indicadores_silver_df = dominio_indicadores_silver_df.withColumnRenamed('SigIndicador', 'SiglaIndicador').withColumnRenamed('DscIndicador', 'DescricaoIndicador')

# Verificar se as colunas foram renomeadas corretamente
print("Dados de Domínio de Indicadores - Após Renomear colunas")
display(dominio_indicadores_silver_df.limit(5))

# DBTITLE 1,Salvar Dados Transformados na Camada Silver
# Verificar os dados antes de salvar
print("Dados da Camada Silver")
display(inadimplencia_silver_df.limit(5))
display(economicos_silver_df.limit(5))
display(dominio_indicadores_silver_df.limit(5))

# Salvar dados diretamente no local correto
inadimplencia_silver_df.write.format('parquet').mode('overwrite').save('dbfs:/mnt/Aneel/Silver/inadimplencia_silver')
economicos_silver_df.write.format('parquet').mode('overwrite').save('dbfs:/mnt/Aneel/Silver/economicos_silver')
dominio_indicadores_silver_df.write.format('parquet').mode('overwrite').save('dbfs:/mnt/Aneel/Silver/dominio_indicadores_silver')

# Verificar os dados após salvar
print("Dados da Camada Silver")
print("======================")
print("Tabela de Inadimplência")
display(spark.read.format('parquet').load('dbfs:/mnt/Aneel/Silver/inadimplencia_silver'))
print("Tabela de indices econômicos")
display(spark.read.format('parquet').load('dbfs:/mnt/Aneel/Silver/economicos_silver'))
print("Tabela de Domínio de Indicadores")
display(spark.read.format('parquet').load('dbfs:/mnt/Aneel/Silver/dominio_indicadores_silver'))

Dados da Camada Bronze
Tabela de Inadimplência


DatGeracaoConjuntoDados,SigAgente,NumCNPJ,SigIndicador,AnoIndice,NumPeriodoIndice,VlrIndiceEnviado
05-07-2024,EQUATORIAL PI,6840748000189,IResBR1,2012,1,"3,54"
05-07-2024,EQUATORIAL PI,6840748000189,IResBR3,2012,1,"1,08"
05-07-2024,EQUATORIAL PI,6840748000189,IResBR6,2012,1,",51"
05-07-2024,EQUATORIAL PI,6840748000189,IResBR12,2012,1,",00"
05-07-2024,EQUATORIAL PI,6840748000189,IResBR18,2012,1,",00"


Tabela de Desemprego


Ano,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,Media
2012,null,null,"8,0","7,8","7,7","7,6","7,5","7,4","7,1","7,0","6,8","6,9","7,4"
2013,"7,3","7,8","8,1","7,9","7,7","7,5","7,4","7,2","7,0","6,8","6,6","6,3","7,3"
2014,"6,5","6,8","7,2","7,2","7,1","6,9","7,0","7,0","6,9","6,7","6,6","6,6","6,9"
2015,"6,9","7,5","8,0","8,1","8,3","8,4","8,7","8,9","9,0","9,1","9,1","9,1","8,4"
2016,"9,6","10,3","11,1","11,3","11,3","11,4","11,7","11,9","11,9","11,9","12,0","12,2","11,4"


Tabela de PIB


...

DatGeracaoConjuntoDados,SigAgente,NumCNPJ,SigIndicador,AnoIndice,NumPeriodoIndice,VlrIndiceEnviado
2024-07-05,CEEE-D,8467115000100,IResBR21,2016,6,1.72
2024-07-05,EMR,19527639000158,IResBR21,2016,6,0.12
2024-07-05,CERPRO,44560381000139,IResBR21,2016,6,0.0
2024-07-05,CERRP,46598678000119,IResBR21,2016,6,0.0
2024-07-05,CERIM,50235449000107,IResBR21,2016,6,1.09
2024-07-05,CELESC,8336783000190,IResBR21,2016,6,1.04
2024-07-05,CERIS,57384943000182,IResBR21,2016,6,0.0
2024-07-05,CEDRAP,60196987000193,IResBR21,2016,6,2.35
2024-07-05,CETRIL,49313653000110,IResBR21,2016,6,0.21
2024-07-05,CERIPA,49606312000132,IResBR21,2016,6,0.0


Tabela de indices econômicos


AnoIndice,TaxaDesemprego,Inflacao,PIB
2012,7.4,344.4,4.8147601E12
2013,7.3,365.8,5.3316189E12
2014,6.9,388.9,5.7789529E12
2015,8.4,424.0,5.9957869E12
2016,11.4,461.1,6.2693279E12
2017,12.9,477.0,6.5854788E12
2018,12.4,494.5,7.0041411E12
2019,12.1,512.9,7.389131E12
2020,13.5,529.4,7.6095968E12
2021,13.5,573.3,9.0121416E12


Tabela de Domínio de Indicadores


SiglaIndicador,DescricaoIndicador
AREA,"Área do conj., expressa em km2, correspondente a área geogr. e não a área elétr."
AREAT,Área do conjunto em km2
CM,Encargo de uso do sistema de distribuição aplicado à unidade cons. (mensal)
CMA,Encargo de uso do sistema de distribuição aplicado à unidade consumidora (anual)
CMM,"Consumo médio mensal, em MWh, média aritmética simples do último ano, excluin-"
CMMT,Consumo médio mensal em MWh
CMMTCO,Energia cons comercial nos últimos 12 meses (ano móvel) MT e BT
CMMTIN,Energia cons industrial nos últimos 12 meses (ano móvel) MT e BT
CMMTOU,Energia cons outras classes nos últimos 12 meses (ano móvel) MT e BT
CMMTRE,Energia cons residencial nos últimos 12 meses (ano móvel) MT e BT
